In [185]:
import requests 
import pandas as pd
import re
import warnings
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

warnings.simplefilter('ignore')
from bs4 import BeautifulSoup as bs

In [10]:
url = f"https://www.autotrader.co.uk/car-search?postcode=sl15pf&radius=1500&include-delivery-option=on&page={page}"
pages = requests.get(url,headers={"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'})
soup = bs(pages.content, "html.parser")

In [11]:
listing_url = soup.find("a", class_ = 'js-click-handler listing-fpa-link tracking-standard-link').get('href')
listing_page = requests.get(f"https://www.autotrader.co.uk{listing_url}",headers={"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'})
listing_soup = bs(listing_page.content, "html.parser")

In [263]:
driver_path = './chromedriver'
driver = webdriver.Chrome(executable_path=driver_path)
driver.get(f"https://www.autotrader.co.uk{listing_url}")

In [264]:
WebDriverWait(driver,20).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@id='sp_message_iframe_687971']")))
WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[aria-label='Accept All']"))).click()

In [265]:
driver.execute_script("window.scrollTo(100,1000);")

In [266]:
span_tag = driver.find_element(By.XPATH, "//*[contains(text(), 'Running costs')]")
span_tag.find_element(By.XPATH, "..").click()

In [271]:
combined_mpg = driver.find_elements(By.XPATH, "//*[contains(text(), 'mpg')]")[-1]

In [272]:
combined_mpg.text

'55.40 mpg'

In [311]:
import requests 
import pandas as pd
import re
import warnings
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

warnings.simplefilter('ignore')
from bs4 import BeautifulSoup as bs

driver_path = './chromedriver'
driver = webdriver.Chrome(executable_path=driver_path)

cols = ['price','make_model','year','type','mileage','engine_size','power','gearbox','fuel_type','number_of_owners','location','combined_mpg']
auto_trader_df = pd.DataFrame(columns=cols)

for page in range(1,9):
    print(f"Page {page}")
    url = f"https://www.autotrader.co.uk/car-search?postcode=sl15pf&radius=1500&include-delivery-option=on&page={page}"
    pages = requests.get(url,headers={"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'})
    soup = bs(pages.content, "html.parser")
    
    price = []
    make_model = []
    year = []
    vehicle_type = []
    mileage = []
    engine_size = []
    power = []
    gearbox = []
    fuel_type = []
    num_of_owners = []
    location = []
    combined_mpg = []

    for item in soup.find_all("li", class_ = 'search-page__result'):
        if len(item.find_all("li", class_ = 'atc-type-picanto--medium')) == 8:
            try:
                price.append(item.find("div", class_ = 'product-card-pricing__price').text.split('\n')[1])
                make_model.append(item.find("h3", class_ = 'product-card-details__title').text.split('\n')[1])
                specs = item.find_all("li", class_ = 'atc-type-picanto--medium')
                location.append(item.find_all("li", class_ = 'product-card-seller-info__spec-item atc-type-picanto')[-1].text.split('\n')[4].title())

                listing_url = item.find("a", class_ = 'js-click-handler listing-fpa-link tracking-standard-link').get('href')
                listing_page = requests.get(f"https://www.autotrader.co.uk{listing_url}",headers={"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'})
                listing_soup = bs(listing_page.content, "html.parser")

                driver.get(f"https://www.autotrader.co.uk{listing_url}")

                WebDriverWait(driver,20).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@id='sp_message_iframe_687971']")))
                WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[aria-label='Accept All']"))).click()

                driver.execute_script("window.scrollTo(100,1200);")
                time.sleep(5)

                span_tag = driver.find_element(By.XPATH, "//*[contains(text(), 'Running costs')]")
                span_tag.find_element(By.XPATH, "..").click()

                time.sleep(5)
                mpg = driver.find_elements(By.XPATH, "//*[contains(text(), 'mpg')]")[-1]

                combined_mpg.append(mpg.text)

                driver.quit()
                
            except:
                pass
            
            try:
                year.append(specs[0].text)
                vehicle_type.append(specs[1].text)
                mileage.append(specs[2].text)
                engine_size.append(specs[3].text)
                power.append(specs[4].text)
                gearbox.append(specs[5].text)
                fuel_type.append(specs[6].text)
                num_of_owners.append(specs[7].text)
            except:
                pass
        else:
            pass
        

    for index in range(len(price)):
        try:
            auto_trader_df = auto_trader_df.append({'price':price[index],'make_model':make_model[index],'year':year[index],'type':vehicle_type[index],'mileage':mileage[index],'engine_size':engine_size[index],'power':power[index],'gearbox':gearbox[index],'fuel_type':fuel_type[index],'number_of_owners':num_of_owners[index],'location':location[index],'combined_mpg':combined_mpg[index]},ignore_index=True)
        except:
            pass
        
auto_trader_df['price'] = auto_trader_df.price.str.replace('£','')
auto_trader_df['price'] = auto_trader_df.price.str.replace(',','')
auto_trader_df['price'] = pd.to_numeric(auto_trader_df['price'])

auto_trader_df['year'] = auto_trader_df['year'].apply(lambda year: year[:4])
auto_trader_df['year'] = pd.to_numeric(auto_trader_df['year'])

auto_trader_df['mileage'] = auto_trader_df['mileage'].apply(lambda mileage: mileage[:-6])
auto_trader_df['mileage'] = auto_trader_df.mileage.str.replace(',','')
auto_trader_df['mileage'] = pd.to_numeric(auto_trader_df['mileage'])

auto_trader_df.dropna(inplace=True)
auto_trader_df.reset_index(inplace=True)
auto_trader_df.drop(columns='index',inplace=True)

# Engine size is shown in Litres
auto_trader_df['engine_size'] = auto_trader_df.engine_size.str.replace('L','')
auto_trader_df['engine_size'] = pd.to_numeric(auto_trader_df['engine_size'])

auto_trader_df['number_of_owners'] = auto_trader_df['number_of_owners'].apply(lambda num_of_owners: num_of_owners[0])
auto_trader_df['number_of_owners'] = pd.to_numeric(auto_trader_df['number_of_owners'], errors='coerce')

# Power will be converted to BHP if the value is in PS units
auto_trader_df['power'] = auto_trader_df['power'].apply(lambda power: int(power[:-2])/1.014 if 'PS' in power else power)
auto_trader_df['power'] = auto_trader_df['power'].apply(lambda power: power[:-3] if 'BHP' in str(power) else power)
auto_trader_df['power'] = pd.to_numeric(auto_trader_df['power'])

auto_trader_df['make'] = auto_trader_df['make_model'].apply(lambda make: make.split()[0])
auto_trader_df['model'] = auto_trader_df['make_model'].apply(lambda make: ' '.join(make.split()[1:]))
auto_trader_df.drop(columns='make_model',inplace=True)

auto_trader_df['combined_mpg'] = auto_trader_df['combined_mpg'].apply(lambda mpg: mpg.split()[0])
auto_trader_df['combined_mpg'] = pd.to_numeric(auto_trader_df['combined_mpg'])

auto_trader_df.dropna(inplace=True)
auto_trader_df.reset_index(inplace=True)
auto_trader_df.drop(columns='index',inplace=True)

for index,make in enumerate(auto_trader_df['make']):
    if re.match(r"Mazda[0-9]+",make):
        auto_trader_df['make'][index] = re.split('(\d+)',make)[0]
        auto_trader_df['model'][index] = re.split('(\d+)',make)[1]
    else:
        pass

Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8


In [312]:
auto_trader_df

,price,year,type,mileage,engine_size,power,gearbox,fuel_type,number_of_owners,location,combined_mpg,make,model
0,999,2010,Hatchback,49000,1.20,68.047337,Manual,Petrol,6.0,Sutton,52.3,Ford,KA
1,650,2004,Hatchback,126000,2.00,74.000000,Manual,Diesel,5.0,Hounslow,44.8,Volkswagen,Golf
2,575,2003,Hatchback,78000,1.20,74.000000,Manual,Petrol,4.0,Romford,38.2,Vauxhall,Corsa
3,875,2006,Hatchback,134000,1.60,110.000000,Manual,Petrol,2.0,Southsea,47.9,Peugeot,307
4,700,2007,Hatchback,124082,1.25,74.000000,Manual,Petrol,5.0,Hornchurch,47.1,Ford,Fiesta
5,2000,2009,Convertible,79000,2.00,143.000000,Automatic,Petrol,4.0,London,34.0,Ford,Focus Cc
6,790,2005,Hatchback,135599,1.40,67.000000,Manual,Diesel,4.0,Reading,64.2,Ford,Fusion
7,3990,2007,Saloon,102987,3.00,194.280079,Automatic,Diesel,4.0,Luton,43.5,BMW,5 Series
8,2500,2006,Hatchback,117000,1.20,55.000000,Manual,Petrol,5.0,Kingston Upon Thames,47.1,Volkswagen,Polo
9,995,2007,Hatchback,110500,1.50,105.000000,Manual,Diesel,5.0,Slough,52.3,Nissan,Qashqai


In [313]:
pd.get_dummies(auto_trader_df['type'])

,Combi Van,Convertible,Coupe,Estate,Hatchback,MPV,SUV,Saloon
0,0,0,0,0,1,0,0,0
1,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0
3,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,0
5,0,1,0,0,0,0,0,0
6,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,1
8,0,0,0,0,1,0,0,0
9,0,0,0,0,1,0,0,0


In [314]:
pd.get_dummies(auto_trader_df['gearbox'])

,Automatic,Manual
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
5,1,0
6,0,1
7,1,0
8,0,1
9,0,1


In [315]:
pd.get_dummies(auto_trader_df['fuel_type'])

,Diesel,Petrol
0,0,1
1,1,0
2,0,1
3,0,1
4,0,1
5,0,1
6,1,0
7,1,0
8,0,1
9,1,0
